# Data Journalism Lesson 22: Beeswarm plots

What if we combined bar charts and scatterplots?

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('seaborn')
await micropip.install("nbformat>=4.2.0")

In [ ]:
from IPython.display import display, HTML
import pandas as pd

# --- Simple Grading/Checking Functions ---
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_df_exists(df, df_name, expected_min_rows=None, expected_cols=None):
    if not isinstance(df, pd.DataFrame) or (df.empty and expected_min_rows != 0):
        display_feedback(False, f'{df_name} DataFrame is not loaded correctly or is unexpectedly empty.', 'Please check the loading or filtering process.')
        return False
    msg_correct = f'{df_name} DataFrame checked.'
    correct = True
    msg_incorrect_list = []

    if expected_min_rows is not None and len(df) < expected_min_rows:
        msg_incorrect_list.append(f' Expected at least {expected_min_rows} rows, got {len(df)}.')
        correct = False
    if expected_cols is not None:
      if not all(col in df.columns for col in expected_cols):
        missing_cols = [col for col in expected_cols if col not in df.columns]
        msg_incorrect_list.append(f' Missing expected columns: {missing_cols}.')
        correct = False
    
    if correct:
        display_feedback(True, msg_correct, '')
    else:
        display_feedback(False, '', ' '.join(msg_incorrect_list))
    return correct

def check_seaborn_params(call_args, expected_params, plot_name):
    # Basic check for key=value pairs in seaborn calls
    correct = True
    messages = []
    for p_name, p_val_expected in expected_params.items():
        p_val_actual = call_args.get(p_name)
        if p_val_actual == p_val_expected:
            messages.append(f'Correct {p_name} for {plot_name}.')
        else:
            correct = False
            messages.append(f'Incorrect {p_name} for {plot_name}. Expected \'{p_val_expected}\', got \'{p_val_actual}\'.')
    
    final_message_correct = f'Key plot parameters for {plot_name} seem correct!'
    final_message_incorrect = ' '.join(messages)
    if not messages:
        display_feedback(True, f'{plot_name} created.', '')
        return
    display_feedback(correct, final_message_correct, final_message_incorrect)

In [ ]:
# --- State Setup and Data Loading ---
default_state_abbr = 'MN'
state_full_name = 'Minnesota'

accredited_colleges_url = "../_static/college-cost/accredited.csv"

accredited_df_initial = pd.read_csv(accredited_colleges_url)
belowpar_df_initial = accredited_df_initial[accredited_df_initial['C150_4'] < 0.5].copy()

# Expected column names for checks
accredited_expected_cols = ['C150_4', 'ACCREDAGENCY', 'SCHTYPE', 'COSTT4_A']

In [ ]:
from myst_nb import glue

glue("state_full_name", state_full_name, display=False)

## The Goal

In this lesson, you'll learn how to create and interpret beeswarm plots, a powerful visualization tool that combines elements of category-based plots and scatterplots to show distributions. By the end of this tutorial, you'll understand when to use beeswarm plots, how to construct them using Seaborn, and how to customize them to reveal patterns in your data. You'll practice these skills using real-world data on college completion rates and costs, gaining insight into how to uncover and communicate trends across multiple categories.

## Why Visualize Data?

As we've moved through learning data visualization basics, you've hopefully picked up on the fact that data visualization lives in this paradox of being centuries old and yet feels like new. There's new data, and new forms of visualization happening all the time, yet we're still learning why it all works. 

In 1984, William S. Cleveland and Robert McGill, two statisticians at Bell Labs, attempted to lay out a theory of graphical perception in the Journal of the American Statistical Association. In short, they wanted a theory of why does this all work? Why do people see the story in data from graphics? And what they wrote remains one of the foundational works to this day.

Cleveland and McGill theorized that there were 10 elementary perceptual tasks that people use to make meaning of a chart. These tasks involve judging position, length, angle, area, volume, slope, and color/shading.

(Note of how 1984 this paper is: They didn't emphasize color as much, likely due to printing costs and limitations at the time, but it's a crucial perceptual task today.)

Their core idea: "A graphical form that involves elementary perceptual tasks that lead to more accurate judgements than another graphical form (with the same quantitative information) will result in a better organization and increase the chances of a correct perception of patterns and behavior."

This helps explain why certain charts work well for specific data types. Bar charts excel at comparing lengths (position on a common scale), while scatterplots leverage position on non-aligned scales to show relationships. 

The form we're using in this chapter is the beeswarm plot (or simply swarm plot in Seaborn). It's the love child of a categorical plot (like a bar chart, showing groups) and a scatterplot (showing individual points). It helps show the distribution of numerical values for different categories. Where a bar chart might show only the average, and a simple scatterplot might have overlapping points, the swarm plot arranges the points within each category to avoid (or minimize) overlap, revealing the density and spread of the data.

That's when you need a beeswarm (or swarm plot).

## The Basics

If you're a college student, there are administrators on your campus worried right now about something called the enrollment cliff. What is this scary sounding thing? It's a topic of intense discussion and debate in higher education and it goes a little something like this: In 2025(ish), we will hit peak high school graduates. The number of high school graduates has been going up for decades. But as of 2025, no longer. It'll drop. By some estimates, there will be 400,000 fewer college-bound high school graduates by 2030. That might not seem like a lot when you figure there's 345 million people in the United States, but that 400,000 student decline would be 4/5ths of the undergrads at the 18 Big Ten campuses *combined*. 

This decline is not going to be felt equally. And it's already being felt. More than 100 schools have closed, merged or been consolidated into other schools since 2016. Hundreds more schools have seen steep declines in enrollment, threatening their existence. At the same time, most flagship campuses (like our aforementioned Big Ten schools) are growing. 

So who is going to feel the pain from the enrollment cliff the most? We don't know exactly, but we can use the data we've been looking at and the swarm plot to raise questions about some schools.

What is a swarm plot (beeswarm)? It's the child of a categorical plot and a scatterplot. It looks like a categorical plot because the data is grouped along one axis based on category. It looks like a scatterplot because every row of data is typically represented as a dot. The key feature is that the dots within each category are adjusted (usually along the axis perpendicular to the category grouping) to prevent overlap, forming characteristic 'swarms'. What is it good for? Swarm plots are excellent at showing the distribution of a quantitative variable across different categories, revealing density, spread, and potential outliers within each group.

Up to this point, we've been using `plotly` for visualization. In this chapter, we're going to explore a new visualization library: `seaborn`. There are many different tools for data visualization, and they each have their own strengths and weaknesses. One of Seaborn's strengths is beeswarm plots. And as you'll see, the way we plot data is very similar to what you've done so far.

Like bar charts, beeswarm work best with a manageable number of categories. Unlike standard scatterplots, they can become crowded if there are very large numbers of points within a category, though Seaborn's implementation handles this reasonably well.

Swarm plots are built into the Seaborn library. We'll also need `pandas` and `matplotlib.pyplot`.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

And now we'll grab some data. This data comes from the U.S. Department of Education's College Scorecard, just like the last two chapters. But this dataset is all main campuses for schools that mostly grant bachelor's degrees or higher. In other words -- colleges and universities. We have both public and private this time. The difference this time -- we're only looking at accredited schools.

Accreditation agencies are designed to ensure a school is meeting certain requirements. To be accredited, schools have to do A, B and C. And then they have to demonstrate that they indeed do A, B and C and do it at a certain level. Accreditation is supposed to give students a sign that this school is legit.

Starting with the Obama administration, the U.S. Department of Education had been highly critical of -- and indeed have closed down -- private, for-profit colleges. Schools that lose accreditation often lose access to federal funding, and students who go there can't take out federally backed student loans or get federal student aid. No money means no college.

There are nearly two dozen accreditors out there, but just seven of them cover the overwhelming majority of schools. This dataset is all colleges and universities accredited by these big seven accreditation agencies.

In [ ]:
accredited_df = pd.read_csv("../_static/college-cost/accredited.csv")

Let's take a quick peek at it to see what we have to work with.

In [ ]:
display(____.head())

Similar to scatterplots, swarm plots often don't require a lot of complex data reshaping. We need one categorical variable and one numerical variable.

The swarm plot uses `sns.swarmplot()`. We need to specify `data`, `x`, and `y`.
- By default, Seaborn often plots the categorical variable on the x-axis and the numerical variable on the y-axis.

### Exercise 1: Your first beeswarm (swarm plot)

Create a swarm plot showing the distribution of completion rates (`C150_4`) for each accreditation agency (`ACCREDAGENCY`).

In [ ]:
# Increase figure size for better readability, especially with long category names
plt.figure(figsize=(8, 10)) 

ax1 = sns.swarmplot(
    data=____,
    x=____,
    y=____,
    size=3
)

ax1.set_title('Completion Rate Distribution by Accreditation Agency')
ax1.set_xlabel('Completion Rate (6-year)')
ax1.set_ylabel('Accreditation Agency')
plt.tight_layout()
plt.show()

What do we see and what are we looking for? What we're looking for here are patterns in the distribution for each agency. Is there a blob of dots (high density) on one side or the other? Does the center of the swarm differ between agencies? 

Look at the Middle States Commission of Higher Education, then look at the Higher Learning Commission, then the Southern Association of Colleges and Schools Commission on Colleges. Note how the densest part of the swarm -- which represents where most schools accredited by that agency fall -- seems to shift? A shift to the left indicates lower completion rates on average for schools under that agency. 

Another surprising thing I'm seeing here is -- how many schools have below 50 percent completion percentages. Find the 0.5 line on the x-axis and look at all the dots left of that. That means fewer than half of first-time, first-year students at those schools graduate in six years. That's bonkers to me -- that finishing college is worse than a coin-flip in these places.

Let's look specifically at them, and make a slightly different kind of plot to draw out some differences. 

First, let's make a new DataFrame called `belowpar_df` containing only schools where the completion rate (`C150_4`) is less than 0.5. The DataFrame `belowpar_df_initial` is already created in the setup cells.

### Exercise 2: Filter for low completion rates

Create `belowpar_df` by filtering `accredited_df`.

In [ ]:
belowpar_df = accredited_df[accredited_df[____] < ____].copy()

display(belowpar_df.head())

Now that we have that slice of data, we're going to introduce a different Seaborn function that's related to `swarmplot`: `stripplot` with jitter enabled. Sometimes, when the data isn't dense enough for `swarmplot`'s algorithm to arrange points neatly, or if you prefer a simpler random scatter, `stripplot` is useful.

`stripplot` draws a scatterplot where one variable is categorical. We'll add `jitter=True` to spread out the points horizontally within each category so they don't all fall on a single vertical line.

### Exercise 3: Using stripplot with jitter

For this chart, instead of using accrediting agencies, we're going to look at the school type -- `SCHTYPE` -- as our grouping variable. Are these schools that fail to graduate more than half of their students primarily public, private non-profit, or private for-profit? Let's find out using `sns.stripplot`. 

Plot `C150_4` (numeric, on the y-axis) against `SCHTYPE` (categorical, on the x-axis). Remember to enable `jitter`.

In [ ]:
plt.figure(figsize=(8, 5))

ax3 = sns.stripplot(
    data=____,
    x=____,
    y=____,
    jitter=____,
    size=4
)

ax3.set_title('Completion Rate (<50%) by School Type')
ax3.set_xlabel('School Type (Numeric)')
ax3.set_ylabel('Completion Rate (6-year)')
plt.tight_layout()
plt.show()

Well ... we got something. But what is a school type of 1? Or 2? Or 3? You have to dig into the documentation (or recall from previous chapters/context) to find it: 1 is public, 2 is private non-profit, and 3 is proprietary (private for-profit).

How do we clean this up a bit? Since Seaborn integrates with Matplotlib, we can use Matplotlib's functions to customize the plot *after* creating it with Seaborn. We'll use `plt.xticks()` to set the tick locations (`[0, 1, 2]`) and their corresponding labels (`["Public", "Private Nonprofit", "Private For-Profit"]`).

In [ ]:
plt.figure(figsize=(8, 5))

ax4 = sns.stripplot(
    data=belowpar_df, 
    x='SCHTYPE',
    y='C150_4', 
    jitter=True,
    size=4
)

# Customize Y-axis ticks and labels
tick_locs = [0, 1, 2]
tick_labels = ["Public", "Private Nonprofit", "Private For-Profit"]
plt.xticks(ticks=____, labels=____) 

ax4.set_title('Completion Rate (<50%) by School Type')
ax4.set_xlabel('School Type') # Update label now that ticks are meaningful
ax4.set_ylabel('Completion Rate (6-year)')
plt.tight_layout()
plt.show()

That's one more way we can add some context to this and bring us back to our original goal of raising questions about colleges in the coming enrollment cliff. How much does it cost to go to one of these schools? Are these schools charging a lot of money and then leaving students without a degree and potentially a lot of debt?

### Exercise 4: Adding color

How can we incorporate cost? By mapping the cost column (`COSTT4_A`) to the color of the points using the `hue` parameter in `sns.stripplot`. Seaborn will automatically apply a color gradient.

Recreate the previous plot, but add `hue='COSTT4_A'`. You might want to specify a sequential color `palette` like `'viridis'`, `'plasma'`, `'magma'`, or `'rocket'`.

In [ ]:
plt.figure(figsize=(10, 6)) # Make wider to accommodate legend

ax5 = sns.stripplot(
    data=____, 
    x=____,
    y=____,
    hue=____,
    jitter=True,
    palette=____, # e.g., 'viridis', 'magma'
    size=5,
    legend='auto'
)

# Customize Y-axis ticks and labels
tick_locs = [0, 1, 2]
tick_labels = ["Public", "Private Nonprofit", "Private For-Profit"]
plt.yticks(ticks=tick_locs, labels=tick_labels) 

ax5.set_title('Completion Rate (<50%) by School Type and Cost')
ax5.set_xlabel('School Type')
ax5.set_ylabel('Completion Rate (6-year)')

# Improve legend if possible (may require manual creation for better labels)
# Example: plt.legend(title='Avg Cost', loc='upper right') # Basic legend adjustment
ax5.legend(title='Avg Cost (COSTT4_A)', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout to prevent legend overlap
plt.show()

Now what do you see? Look at the colors within each school type category.
- **Public schools (<50% completion):** Mostly darker colors, indicating lower costs.
- **Private Nonprofit schools (<50% completion):** A wider range of colors, but quite a few lighter dots, indicating higher costs compared to the publics in this group.
- **Private For-Profit schools (<50% completion):** Also seem to have higher costs than the publics, with many in the mid-to-lighter color range.

A question you could ask education experts from this: What does the future hold for under-performing schools with large tuition bills, especially private ones? A reasonable guess might be: not good things, particularly as enrollment pressures increase.

## The Recap

Throughout this lesson, you've mastered the creation and interpretation of swarm plots and jittered strip plots using Seaborn, versatile tools for visualizing distributions across categories. You've learned to construct basic plots (`sns.swarmplot`, `sns.stripplot`), customize axes for clarity (`plt.yticks`), and enhance them by mapping a third variable to color (`hue`) to add another dimension of data. Remember, these types of plots are particularly effective when you need to show how individual data points are distributed within and across categories, making them ideal for comparing groups or identifying patterns related to density and spread.

## Terms To Know

- **Beeswarm plot / Swarm plot**: A data visualization that plots individual data points along a numerical axis, grouped by category on the other axis, arranging points within each category to avoid overlap and show distribution density.
- **Seaborn**: A Python data visualization library based on Matplotlib that provides a high-level interface for drawing attractive and informative statistical graphics.
- **`seaborn.swarmplot()` (or `sns.swarmplot()`)**: The Seaborn function for creating swarm plots.
- **Strip plot**: A scatter plot where one variable is categorical. Often used with jitter.
- **`seaborn.stripplot()` (or `sns.stripplot()`)**: The Seaborn function for creating strip plots.
- **`jitter=True`**: An argument for `sns.stripplot()` that adds random noise to the position of points along the categorical axis to reduce overlap.
- **`hue`**: A parameter in many Seaborn functions used to map a third variable to the color of plot elements.
- **`palette`**: A parameter in Seaborn used to specify the color map or sequence for plot elements, especially when using `hue`.
- **`matplotlib.pyplot` (as `plt`)**: A collection of functions that make Matplotlib work like MATLAB. Often used alongside Seaborn for figure-level customization (like setting figure size or axis ticks/labels).
- **`plt.yticks()` / `plt.xticks()`**: Matplotlib functions to get or set the tick locations and labels for the y-axis or x-axis, respectively.